# Getting Started with PCB Defect Detection

This tutorial will walk you through the basics of using the PCB Defect Detection framework.

## What You'll Learn

1. Loading and using pre-trained models
2. Creating foundation model adapters
3. Running inference on PCB images
4. Understanding the results
5. Training your own model

## Prerequisites

Make sure you have installed the required dependencies:

```bash
pip install -r requirements.txt
```

## Step 1: Import Required Libraries

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torchvision.transforms as transforms
from pathlib import Path

# Import our custom modules
from enhanced_pcb_model import create_enhanced_model
from core.foundation_adapter import create_foundation_adapter, set_reproducible_seed

# Set reproducible seed for consistent results
set_reproducible_seed(42)

print("✓ All imports successful!")
print(f"PyTorch version: {torch.__version__}")

# Determine device
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Using device: {device}")

## Step 2: Create and Load a Model

Let's start by creating an enhanced PCB model with LoRA adaptation:

In [ ]:
# Create enhanced PCB model
model, loss_fn = create_enhanced_model(
    num_classes=5,
    backbone='resnet50',
    lora_rank=4
)

model.to(device)
model.eval()

# Display model information
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
efficiency = 1.0 - (trainable_params / total_params)

print(f"Model created successfully!")
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Parameter efficiency: {efficiency:.4f} ({efficiency*100:.2f}% frozen)")

# Define class names
class_names = ['normal', 'missing_component', 'solder_bridge', 'misalignment', 'short_circuit']

## Step 3: Prepare Image Preprocessing

In [ ]:
# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Helper function to preprocess images
def preprocess_image(image_path):
    """Load and preprocess an image for inference"""
    image = Image.open(image_path).convert('RGB')
    return transform(image).unsqueeze(0).to(device), image

print("✓ Image preprocessing pipeline ready")

## Step 4: Create Synthetic Test Data

Since we might not have actual PCB images, let's create some synthetic test data:

In [ ]:
# Create synthetic PCB images for demonstration
def create_synthetic_pcb_image(defect_type='normal', size=(224, 224)):
    """Create a synthetic PCB image with simulated defects"""
    
    # Base PCB (green background)
    image = np.full((*size, 3), [0, 100, 0], dtype=np.uint8)
    
    # Add circuit traces (copper color)
    for i in range(0, size[0], 20):
        image[i:i+2, :] = [184, 115, 51]  # Copper color
    
    for j in range(0, size[1], 30):
        image[:, j:j+2] = [184, 115, 51]  # Copper color
    
    # Add components (black rectangles)
    component_positions = [(50, 50), (100, 100), (150, 150), (80, 170)]
    
    for x, y in component_positions:
        if defect_type == 'missing_component' and (x, y) == component_positions[0]:
            continue  # Skip first component to simulate missing
        
        # Draw component
        image[y:y+15, x:x+25] = [20, 20, 20]  # Black component
        
        # Add solder bridge if needed
        if defect_type == 'solder_bridge' and (x, y) == component_positions[1]:
            image[y+15:y+25, x:x+25] = [200, 200, 200]  # Silver solder bridge
    
    # Add misalignment
    if defect_type == 'misalignment':
        x, y = component_positions[2]
        image[y+5:y+20, x+10:x+35] = [20, 20, 20]  # Misaligned component
    
    # Add short circuit (red line)
    if defect_type == 'short_circuit':
        image[120:125, 60:140] = [255, 0, 0]  # Red short circuit
    
    return Image.fromarray(image)

# Create sample images for each defect type
sample_images = {}
for defect_type in class_names:
    sample_images[defect_type] = create_synthetic_pcb_image(defect_type)

# Display the synthetic images
fig, axes = plt.subplots(1, 5, figsize=(15, 3))
for i, (defect_type, image) in enumerate(sample_images.items()):
    axes[i].imshow(image)
    axes[i].set_title(defect_type.replace('_', ' ').title())
    axes[i].axis('off')

plt.tight_layout()
plt.show()

print("✓ Synthetic PCB images created")

## Step 5: Run Inference

Now let's test our model on the synthetic images:

In [ ]:
def predict_defect(image, model, transform, class_names, device):
    """Predict defect type from image"""
    
    # Preprocess image
    if isinstance(image, Image.Image):
        image_tensor = transform(image).unsqueeze(0).to(device)
    else:
        image_tensor = image
    
    # Run inference
    with torch.no_grad():
        outputs = model(image_tensor)
        probabilities = torch.softmax(outputs, dim=1)
        predicted_class_id = torch.argmax(probabilities, dim=1).item()
        confidence = probabilities[0][predicted_class_id].item()
    
    return {
        'predicted_class': class_names[predicted_class_id],
        'class_id': predicted_class_id,
        'confidence': confidence,
        'probabilities': {class_names[i]: float(probabilities[0][i]) for i in range(len(class_names))}
    }

# Test on synthetic images
results = {}
for defect_type, image in sample_images.items():
    result = predict_defect(image, model, transform, class_names, device)
    results[defect_type] = result
    
    print(f"\n{defect_type.upper()} PCB:")
    print(f"  Predicted: {result['predicted_class']} (confidence: {result['confidence']:.3f})")
    print(f"  Top probabilities:")
    sorted_probs = sorted(result['probabilities'].items(), key=lambda x: x[1], reverse=True)[:3]
    for class_name, prob in sorted_probs:
        print(f"    {class_name}: {prob:.3f}")

## Step 6: Visualize Results

Let's create a comprehensive visualization of our results:

In [ ]:
# Create visualization of predictions
fig, axes = plt.subplots(2, 5, figsize=(20, 8))

for i, (defect_type, image) in enumerate(sample_images.items()):
    result = results[defect_type]
    
    # Display image
    axes[0, i].imshow(image)
    axes[0, i].set_title(f'Ground Truth: {defect_type.replace("_", " ").title()}')
    axes[0, i].axis('off')
    
    # Display prediction probabilities
    probs = list(result['probabilities'].values())
    colors = ['green' if class_names[j] == result['predicted_class'] else 'lightblue' for j in range(len(class_names))]
    
    axes[1, i].bar(range(len(class_names)), probs, color=colors)
    axes[1, i].set_title(f'Predicted: {result["predicted_class"]} ({result["confidence"]:.2f})')
    axes[1, i].set_xticks(range(len(class_names)))
    axes[1, i].set_xticklabels([name.replace('_', '\n') for name in class_names], rotation=0, fontsize=8)
    axes[1, i].set_ylabel('Probability')
    axes[1, i].set_ylim(0, 1)

plt.tight_layout()
plt.show()

print("\n✓ Visualization complete!")

## Step 7: Explore Foundation Model Adapter

Now let's explore the foundation model adapter capabilities:

In [ ]:
# Create foundation model adapter (if CLIP is available)
try:
    adapter = create_foundation_adapter(
        method="AD-CLIP",
        domain="materials",  # PCBs are materials
        rank=4,
        alpha=32
    )
    
    print(f"Foundation adapter created successfully!")
    print(f"Method: {adapter.config.method}")
    print(f"Domain: {adapter.config.domain}")
    print(f"Parameter efficiency: {adapter.efficiency_ratio:.4f}")
    print(f"Trainable parameters: {adapter.trainable_parameters:,}")
    print(f"Domain prompts: {len(adapter.config.domain_prompts)}")
    
    for i, prompt in enumerate(adapter.config.domain_prompts[:3]):
        print(f"  {i+1}. {prompt}")
    
except Exception as e:
    print(f"Foundation adapter creation failed (likely missing CLIP): {e}")
    print("This is normal if you haven't installed the CLIP dependencies.")

## Step 8: Model Training Simulation

Let's simulate a brief training session to show how the framework works:

In [ ]:
# Create a simple training simulation
from torch.utils.data import DataLoader, TensorDataset

def create_synthetic_dataset(num_samples=100):
    """Create synthetic training data"""
    images = torch.randn(num_samples, 3, 224, 224)
    labels = torch.randint(0, 5, (num_samples,))
    return TensorDataset(images, labels)

# Create datasets
train_dataset = create_synthetic_dataset(200)
val_dataset = create_synthetic_dataset(50)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Setup optimizer (only for trainable parameters)
optimizer = torch.optim.AdamW(
    [p for p in model.parameters() if p.requires_grad], 
    lr=1e-3, 
    weight_decay=1e-4
)

# Quick training simulation (2 epochs)
model.train()
training_history = []

for epoch in range(2):
    total_loss = 0
    num_batches = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        num_batches += 1
    
    avg_loss = total_loss / num_batches
    training_history.append(avg_loss)
    
    print(f"Epoch {epoch + 1}: Average Loss = {avg_loss:.4f}")

# Validation
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"\nValidation Accuracy: {accuracy:.2f}%")

# Plot training curve
plt.figure(figsize=(8, 4))
plt.plot(training_history, 'b-', label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Progress')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("\n✓ Training simulation complete!")

## Step 9: Performance Analysis

Let's analyze the model's performance characteristics:

In [ ]:
import time

# Benchmark inference speed
model.eval()
sample_input = torch.randn(1, 3, 224, 224).to(device)

# Warm up
for _ in range(10):
    with torch.no_grad():
        _ = model(sample_input)

# Benchmark
num_runs = 100
start_time = time.time()

for _ in range(num_runs):
    with torch.no_grad():
        _ = model(sample_input)

end_time = time.time()
avg_inference_time = (end_time - start_time) / num_runs * 1000  # ms

# Calculate model size
def get_model_size(model):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    return param_size / 1024 / 1024  # MB

model_size_mb = get_model_size(model)

# Performance summary
print("📊 Performance Analysis")
print("=" * 50)
print(f"Model Size: {model_size_mb:.2f} MB")
print(f"Total Parameters: {total_params:,}")
print(f"Trainable Parameters: {trainable_params:,} ({(trainable_params/total_params)*100:.2f}%)")
print(f"Parameter Efficiency: {efficiency:.4f}")
print(f"Average Inference Time: {avg_inference_time:.2f} ms")
print(f"Throughput: {1000/avg_inference_time:.1f} images/second")
print(f"Device: {device}")

# Create performance visualization
metrics = ['Model Size (MB)', 'Inference Time (ms)', 'Parameter Efficiency (%)']
values = [model_size_mb, avg_inference_time, efficiency * 100]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Performance metrics bar chart
ax1.bar(metrics, values, color=['skyblue', 'lightcoral', 'lightgreen'])
ax1.set_title('Model Performance Metrics')
ax1.set_ylabel('Value')

# Parameter distribution pie chart
frozen_params = total_params - trainable_params
ax2.pie([trainable_params, frozen_params], 
        labels=[f'Trainable\n({trainable_params:,})', f'Frozen\n({frozen_params:,})'],
        colors=['lightcoral', 'lightblue'],
        autopct='%1.1f%%')
ax2.set_title('Parameter Distribution')

plt.tight_layout()
plt.show()

print("\n✅ Tutorial completed successfully!")

## Summary

In this tutorial, you learned:

1. **Model Creation**: How to create enhanced PCB models with LoRA adaptation
2. **Inference**: How to run predictions on PCB images
3. **Visualization**: How to visualize results and model predictions
4. **Foundation Adapters**: How to use CLIP-based adapters for domain-specific tasks
5. **Training**: How the training process works (simulated)
6. **Performance**: How to analyze model performance and efficiency

## Next Steps

- Explore the [Advanced Tutorials](advanced_tutorials.ipynb)
- Learn about [Hyperparameter Optimization](hyperparameter_optimization.ipynb)
- Try the [Production Deployment Guide](../deployment/production.md)
- Read about the [Research Methodology](../research/methodology.md)

## Resources

- [API Reference](../api/core.rst)
- [User Guide](../user_guide/foundation_models.md)
- [GitHub Repository](https://github.com/soumyajitghosh/pcb-defect-detection)

Happy detecting! 🔍